In [121]:
import gzip
import csv
import json
import pandas as pd
from datetime import datetime
#with gzip.open("C:/Users/varsh/Desktop/TwoSixCapital/lynx_transaction_data.csv.gz", 'rb') as f:
#file_content = f.read()
df = pd.read_csv("C:/Users/varsh/Desktop/TwoSixCapital/lynx_transaction_data.csv.gz", compression='gzip', header=0, sep='|', quotechar='"')
dfa=df
dfa['CohortYear'] = dfa.purchase_year
dfa.drop('purchase_year', axis=1, inplace=True)
dfa['transaction_date']=pd.to_datetime(dfa['transaction_date'])
dfa['PurchaseYear'] = dfa['transaction_date'].apply(lambda x: x.year)
print(dfa.head())

   id            base_license       product_group country product_type  \
0  13  09-31001-1Z9110-YY0377     FloatingLicense      DE         Main   
1  29  09-31001-199X35-Y9135W  LimitedSiteLicense      AT       Update   
2  45  09-31001-W13903-73X75Y       AddOn Channel      DE       Update   
3  61  09-31001-W13903-73X75Y       AddOn Channel      DE       Update   
4  77  09-31001-W390Y9-Z10W5X       AddOn Channel      DE       Update   

   billings transaction_date  CohortYear  PurchaseYear  
0     300.0       2006-07-24        2006          2006  
1     569.0       2010-12-13        2006          2010  
2     179.0       2009-01-15        2006          2009  
3     169.0       2013-11-28        2006          2013  
4     189.0       2013-02-14        2006          2013  


In [122]:
dfa.set_index('base_license', inplace=True)
dfa.reset_index(inplace=True)
dfa.head()

,base_license,id,product_group,country,product_type,billings,transaction_date,CohortYear,PurchaseYear
0,09-31001-1Z9110-YY0377,13,FloatingLicense,DE,Main,300.0,2006-07-24,2006,2006
1,09-31001-199X35-Y9135W,29,LimitedSiteLicense,AT,Update,569.0,2010-12-13,2006,2010
2,09-31001-W13903-73X75Y,45,AddOn Channel,DE,Update,179.0,2009-01-15,2006,2009
3,09-31001-W13903-73X75Y,61,AddOn Channel,DE,Update,169.0,2013-11-28,2006,2013
4,09-31001-W390Y9-Z10W5X,77,AddOn Channel,DE,Update,189.0,2013-02-14,2006,2013


In [123]:
import numpy as np
grouped = dfa.groupby(['PurchaseYear','CohortYear'])

cohorts = grouped.agg({'base_license': pd.Series.nunique,
                       'billings': np.sum})

cohorts.rename(columns={'base_license': 'TotalUsers',
                       'billings': 'Revenue'}, inplace=True)
cohorts.head()

TotalUsers     Revenue
PurchaseYear CohortYear                        
2006         2006               164  9148327.76
2007         2006                63    32693.50
             2007              1736  1730084.75
2008         2006                43    29553.37
             2007               499   328599.09

In [124]:
cohorts.reset_index(inplace=True)
cohorts.set_index(['PurchaseYear','CohortYear'], inplace=True)

cohort_group_size = cohorts['TotalUsers'].groupby(level=0).first()
cohort_group_size.head()

PurchaseYear
2006    164
2007     63
2008     43
2009     45
2010     50
Name: TotalUsers, dtype: int64

In [125]:
cohorts['TotalUsers'].unstack(0)

PurchaseYear,2006,2007,2008,2009,2010,2011,2012,2013
CohortYear,,,,,,,,
2006,164.0,63.0,43.0,45.0,50.0,42.0,47.0,47.0
2007,NaN,1736.0,499.0,747.0,655.0,590.0,636.0,535.0
2008,NaN,NaN,12674.0,4786.0,4497.0,4192.0,4208.0,3680.0
2009,NaN,NaN,NaN,23302.0,7498.0,7931.0,7855.0,6887.0
2010,NaN,NaN,NaN,NaN,37237.0,11103.0,12457.0,10821.0
2011,NaN,NaN,NaN,NaN,NaN,52914.0,15879.0,15479.0
2012,NaN,NaN,NaN,NaN,NaN,NaN,56479.0,14959.0
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60629.0


In [126]:
cohorts['Revenue'].unstack(0)

PurchaseYear,2006,2007,2008,2009,2010,2011,2012,2013
CohortYear,,,,,,,,
2006,9148327.76,32693.50,29553.37,43394.57,5.638535e+04,5.219652e+04,6.288294e+04,6.467341e+04
2007,NaN,1730084.75,328599.09,479571.27,5.082186e+05,5.337232e+05,6.427002e+05,5.544030e+05
2008,NaN,NaN,11097448.86,2598329.90,2.965023e+06,3.266833e+06,3.486782e+06,3.298678e+06
2009,NaN,NaN,NaN,22437579.99,4.452721e+06,5.595971e+06,6.387963e+06,6.061836e+06
2010,NaN,NaN,NaN,NaN,3.967539e+07,7.089751e+06,9.501798e+06,8.999460e+06
2011,NaN,NaN,NaN,NaN,NaN,6.206219e+07,1.048262e+07,1.158573e+07
2012,NaN,NaN,NaN,NaN,NaN,NaN,6.049597e+07,9.730164e+06
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.242343e+07


In [129]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C:/Users/varsh/Desktop/TwoSixCapital/CohortTriangle.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
title = pd.DataFrame({'Title':'Customers by Cohort triangle'}, index=[0])
title.to_excel(writer, sheet_name='Cohort triangle', startrow=0, index=False, columns = ['Title'], header = False)
cohorts['TotalUsers'].unstack(0).to_excel(writer, sheet_name='Cohort triangle', startrow=2, header = True)

title = pd.DataFrame({'Title':'Revenue by Cohort triangle'}, index=[0])
title.to_excel(writer, sheet_name='Cohort triangle', startrow=12, startcol=0, index=False, columns = ['Title'], header = False)
cohorts['Revenue'].unstack(0).to_excel(writer, sheet_name='Cohort triangle', startrow=14, startcol=0, header = True)

writer.save()